In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.types import TimestampType

In [0]:
%run ../Utils/DBInstanceUtils

In [0]:
%run ../Utils/ProcessingUtils

In [0]:
dbi = DBInstanceUtils('treinamento_formula1')
filename    = 'pitstopsRaw'
extension   = 'delta'
file        = f'{filename}.{extension}'

DBInstanceUtils.set_dbfs_filepath(dbi, 'mnt',f'formula1/bronze/pitstops', file)

In [0]:
dataframe = (spark.read
             .format('delta')
             .load(dbi.get_dbfs_filepath())
             .drop('url'))

dataframe = (Refine(dataframe)
             .camelCase_to_snake_case()
             .add_constcol('ingestion_date', TimestampType(),current_timestamp())
             .load())

In [0]:
filename = 'pitstopsTrusted'
extension = 'delta'
file = filename + '.' + extension
DBInstanceUtils.set_dbfs_savepath(dbi,'mnt','formula1/prata/pitstops',file)

In [0]:
dataframe.write.mode('overwrite').save(dbi.get_dbfs_savepath())